In [3]:
import os, duckdb

base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\ratings_raw"
out_path = os.path.join(base, "noteRatings_master.parquet")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='2GB';")
con.execute(f"PRAGMA temp_directory='{base}';")

print("🚀 Combining all noteRatings parquet files...")
con.execute(f"""
COPY (
  SELECT * FROM read_parquet('{base}\\noteRatings-*.parquet')
) TO '{out_path}' (FORMAT PARQUET);
""")

con.close()
print(f"✅ Master parquet ready → {out_path}")


🚀 Combining all noteRatings parquet files...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Master parquet ready → C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\ratings_raw\noteRatings_master.parquet


In [4]:
import duckdb
import os

# Path to the master parquet file
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\ratings_raw"
master_path = os.path.join(base, "noteRatings_master.parquet")

print(f"📊 Loading and analyzing: {master_path}\n")

con = duckdb.connect()

# Get row count
row_count = con.execute(f"SELECT COUNT(*) FROM '{master_path}'").fetchone()[0]
print(f"Total rows: {row_count:,}\n")

# Get column names and types
print("Column names and types:")
schema = con.execute(f"DESCRIBE SELECT * FROM '{master_path}'").fetchdf()
print(schema.to_string(index=False))
print(f"\nTotal columns: {len(schema)}\n")

# Get basic statistics
print("Preview of first 5 rows:")
preview = con.execute(f"SELECT * FROM '{master_path}' LIMIT 5").fetchdf()
print(preview)

con.close()

📊 Loading and analyzing: C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\ratings_raw\noteRatings_master.parquet

Total rows: 171,877,798

Column names and types:
                         column_name column_type null  key default extra
                              noteId      BIGINT  YES None    None  None
                  raterParticipantId     VARCHAR  YES None    None  None
                     createdAtMillis      BIGINT  YES None    None  None
                             version      BIGINT  YES None    None  None
                               agree      BIGINT  YES None    None  None
                            disagree      BIGINT  YES None    None  None
                             helpful      BIGINT  YES None    None  None
                          notHelpful      BIGINT  YES None    None  None
                    helpfulnessLevel     VARCHAR  YES None    None  None
                        helpfulOther 

In [5]:
# Path to the master parquet file
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data"
master_path = os.path.join(base, "scored_notes_2dim.parquet")

print(f"📊 Loading and analyzing: {master_path}\n")

con = duckdb.connect()

# Get row count
row_count = con.execute(f"SELECT COUNT(*) FROM '{master_path}'").fetchone()[0]
print(f"Total rows: {row_count:,}\n")

# Get column names and types
print("Column names and types:")
schema = con.execute(f"DESCRIBE SELECT * FROM '{master_path}'").fetchdf()
print(schema.to_string(index=False))
print(f"\nTotal columns: {len(schema)}\n")

# Get basic statistics
print("Preview of first 5 rows:")
preview = con.execute(f"SELECT * FROM '{master_path}' LIMIT 5").fetchdf()
print(preview)

con.close()

📊 Loading and analyzing: C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\scored_notes_2dim.parquet

Total rows: 2,339,700

Column names and types:
                      column_name column_type null  key default extra
                           noteId      BIGINT  YES None    None  None
                coreNoteIntercept       FLOAT  YES None    None  None
                  coreNoteFactor1       FLOAT  YES None    None  None
                finalRatingStatus     VARCHAR  YES None    None  None
                         firstTag     VARCHAR  YES None    None  None
                        secondTag     VARCHAR  YES None    None  None
                  coreActiveRules     VARCHAR  YES None    None  None
                 activeFilterTags     VARCHAR  YES None    None  None
                   classification     VARCHAR  YES None    None  None
                  createdAtMillis      DOUBLE  YES None    None  None
            

In [8]:
import os, gc, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data"
ratings_dir = os.path.join(base, "ratings_raw")
scores_path  = os.path.join(base, "scored_notes_2dim.parquet")

output_dir = os.path.join(base, "intermediates")
os.makedirs(output_dir, exist_ok=True)
TEMP_DIR = os.path.join(output_dir, "tmp_join")
os.makedirs(TEMP_DIR, exist_ok=True)

out_path = os.path.join(output_dir, "ratings_with_scores.parquet")

# --- Settings ---
MEM_LIMIT_GB = 4
BATCH_SIZE   = 5_000_000  # tune to fit your RAM

# --- Connect and count total rows across all parquet partitions ---
con = duckdb.connect()
con.execute(f"PRAGMA memory_limit='{MEM_LIMIT_GB}GB';")
con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")

total_rows = con.execute(
    f"SELECT SUM(row_count) FROM (SELECT COUNT(*) AS row_count FROM read_parquet('{ratings_dir}\\noteRatings-*.parquet'))"
).fetchone()[0]

print(f"Total rating rows: {total_rows:,}")
con.close()

# --- Process in batches ---
offset = 0
batch_idx = 0
part_files = []

while offset < total_rows:
    batch_idx += 1
    out_part = os.path.join(TEMP_DIR, f"ratings_scores_part_{batch_idx:03d}.parquet")
    print(f"\n🚀 Joining batch {batch_idx} (offset {offset:,})")

    con = duckdb.connect()
    con.execute(f"PRAGMA memory_limit='{MEM_LIMIT_GB}GB';")
    con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")

    # build query
    query = f"""
    COPY (
        WITH r AS (
            SELECT
                noteId,
                raterParticipantId,
                createdAtMillis,
                helpful,
                notHelpful
            FROM read_parquet('{ratings_dir}\\noteRatings-*.parquet')
            LIMIT {BATCH_SIZE} OFFSET {offset}
        ),
        r2 AS (
            SELECT
                *,
                CASE
                    WHEN helpful = 1 THEN 'HELPFUL'
                    WHEN notHelpful = 1 THEN 'NOT_HELPFUL'
                    ELSE 'UNKNOWN'
                END AS ratingStatus
            FROM r
        )
        SELECT
            r2.raterParticipantId,
            r2.noteId,
            r2.ratingStatus,
            r2.createdAtMillis,
            s.coreNoteIntercept,
            s.coreNoteFactor1
        FROM r2
        LEFT JOIN read_parquet('{scores_path}') s USING (noteId)
    ) TO '{out_part}' (FORMAT PARQUET);
    """

    con.execute(query)
    con.close()
    part_files.append(out_part)
    offset += BATCH_SIZE
    gc.collect()

# --- Merge all parts into one big file ---
print("\n🧩 Combining all parts into one parquet...")
con = duckdb.connect()
con.execute(f"PRAGMA memory_limit='{MEM_LIMIT_GB}GB';")
con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")

con.execute(f"""
COPY (
  SELECT * FROM read_parquet({part_files})
) TO '{out_path}' (FORMAT PARQUET);
""")

con.close()
print(f"✅ Done. ratings_with_scores.parquet written to {out_path}")



Total rating rows: 171,877,798

🚀 Joining batch 1 (offset 0)

🚀 Joining batch 2 (offset 5,000,000)

🚀 Joining batch 2 (offset 5,000,000)

🚀 Joining batch 3 (offset 10,000,000)

🚀 Joining batch 3 (offset 10,000,000)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


🚀 Joining batch 4 (offset 15,000,000)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


🚀 Joining batch 5 (offset 20,000,000)


...


🚀 Joining batch 35 (offset 170,000,000)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


🧩 Combining all parts into one parquet...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Done. ratings_with_scores.parquet written to C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\intermediates\ratings_with_scores.parquet


In [9]:
# Path to the master parquet file
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\intermediates"
master_path = os.path.join(base, "ratings_with_scores.parquet")

print(f"📊 Loading and analyzing: {master_path}\n")

con = duckdb.connect()

# Get row count
row_count = con.execute(f"SELECT COUNT(*) FROM '{master_path}'").fetchone()[0]
print(f"Total rows: {row_count:,}\n")

# Get column names and types
print("Column names and types:")
schema = con.execute(f"DESCRIBE SELECT * FROM '{master_path}'").fetchdf()
print(schema.to_string(index=False))
print(f"\nTotal columns: {len(schema)}\n")

# Get basic statistics
print("Preview of first 5 rows:")
preview = con.execute(f"SELECT * FROM '{master_path}' LIMIT 5").fetchdf()
print(preview)

con.close()

📊 Loading and analyzing: C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\intermediates\ratings_with_scores.parquet

Total rows: 171,877,798

Column names and types:
       column_name column_type null  key default extra
raterParticipantId     VARCHAR  YES None    None  None
            noteId      BIGINT  YES None    None  None
      ratingStatus     VARCHAR  YES None    None  None
   createdAtMillis      BIGINT  YES None    None  None
 coreNoteIntercept       FLOAT  YES None    None  None
   coreNoteFactor1       FLOAT  YES None    None  None

Total columns: 6

Preview of first 5 rows:
                                  raterParticipantId               noteId  \
0  C784F04F26E124F4D6EC01658D8F5565005D3092741FB3...  1714589572200304695   
1  C784F04F26E124F4D6EC01658D8F5565005D3092741FB3...  1727529309810655291   
2  C784F04F26E124F4D6EC01658D8F5565005D3092741FB3...  1728563772543910115   
3  C784F04F26E124F4D6EC0165

In [2]:
import os, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data"
master_path = os.path.join(base, "user_period_master.parquet")
aggregates_path = os.path.join(base, "intermediates", "user_biweekly_aggregates.parquet")
output_path = os.path.join(base, "user_period_master_enriched.parquet")

print("🚀 Enriching user_period_master with biweekly aggregates...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Join and enrich the master dataset
query = f"""
COPY (
    SELECT 
        m.*,
        -- Add biweekly aggregate columns
        a.helpful_count,
        a.not_helpful_count,
        a.unknown_count,
        a.helpful_ratio,
        a.not_helpful_ratio,
        a.avg_core_note_intercept,
        a.min_core_note_intercept,
        a.max_core_note_intercept,
        a.stddev_core_note_intercept,
        a.non_null_intercept_count,
        a.avg_core_note_factor1,
        a.min_core_note_factor1,
        a.max_core_note_factor1,
        a.stddev_core_note_factor1,
        a.non_null_factor1_count
    FROM read_parquet('{master_path}') m
    LEFT JOIN read_parquet('{aggregates_path}') a 
        ON m.user_id = a.raterParticipantId 
        AND m.period_start = a.period_start
    ORDER BY m.user_id, m.period_start
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing join and enrichment query...")
con.execute(query)

print(f"\n✅ Done! Enriched master dataset saved to:\n{output_path}")

# Verify the join
print("\n📊 Verification:")
stats = con.execute(f"""
    SELECT 
        COUNT(*) as total_rows,
        COUNT(helpful_count) as rows_with_aggregate_data,
        ROUND(COUNT(helpful_count) * 100.0 / COUNT(*), 2) as pct_with_aggregate_data,
        COUNT(DISTINCT user_id) as unique_users
    FROM '{output_path}'
""").fetchdf()
print(stats.to_string(index=False))

# Preview the enriched data
print("\n📝 Preview of enriched dataset:")
preview = con.execute(f"""
    SELECT 
        *
    FROM '{output_path}'
    WHERE helpful_count IS NOT NULL
    LIMIT 10
""").fetchdf()
print(preview.to_string(index=False))

con.close()

print("\n✅ Enrichment complete!")

🚀 Enriching user_period_master with biweekly aggregates...

Executing join and enrichment query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! Enriched master dataset saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\user_period_master_enriched.parquet

📊 Verification:
 total_rows  rows_with_aggregate_data  pct_with_aggregate_data  unique_users
   20869319                         0                      0.0       1278621

📝 Preview of enriched dataset:
Empty DataFrame
Columns: [user_id, period_start, ratings_given, notes_rated_unique, tweets_rated_unique, agree_rate, disagree_rate, helpful_rate, somewhat_helpful_rate, not_helpful_rate, avg_helpful_flags, avg_not_helpful_flags, notes_written, num_misleading, num_not_misleading, avg_trustworthySources, media_note_ratio, misleading_flag_sum, not_misleading_flag_sum, share_misleading, avg_flags_per_note, notes_with_status, notes_helpful, notes_not_helpful, notes_nmr, notes_locked, first_helpful, latest_helpful, latest_not_helpful, avg_days_to_first_nonNMR, avg_days_to_lock, share_helpful, share

In [3]:
import os, duckdb

# Debug: Check why join is failing
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data"
master_path = os.path.join(base, "user_period_master.parquet")
aggregates_path = os.path.join(base, "intermediates", "user_biweekly_aggregates.parquet")

con = duckdb.connect()

print("🔍 DEBUGGING JOIN ISSUE\n")
print("=" * 80)

# 1. Check column names and types in master
print("\n1️⃣ user_period_master.parquet columns:")
master_schema = con.execute(f"DESCRIBE SELECT * FROM '{master_path}'").fetchdf()
print(master_schema[master_schema['column_name'].isin(['user_id', 'period_start'])].to_string(index=False))

# 2. Check column names and types in aggregates
print("\n2️⃣ user_biweekly_aggregates.parquet columns:")
agg_schema = con.execute(f"DESCRIBE SELECT * FROM '{aggregates_path}'").fetchdf()
print(agg_schema[agg_schema['column_name'].isin(['raterParticipantId', 'period_start'])].to_string(index=False))

# 3. Sample values from master
print("\n3️⃣ Sample values from user_period_master:")
master_sample = con.execute(f"""
    SELECT user_id, period_start, typeof(user_id) as user_id_type, typeof(period_start) as period_type
    FROM '{master_path}' 
    LIMIT 5
""").fetchdf()
print(master_sample.to_string(index=False))

# 4. Sample values from aggregates
print("\n4️⃣ Sample values from user_biweekly_aggregates:")
agg_sample = con.execute(f"""
    SELECT raterParticipantId, period_start, typeof(raterParticipantId) as user_id_type, typeof(period_start) as period_type
    FROM '{aggregates_path}' 
    LIMIT 5
""").fetchdf()
print(agg_sample.to_string(index=False))

# 5. Check for overlap
print("\n5️⃣ Checking for user_id overlap:")
overlap = con.execute(f"""
    SELECT 
        COUNT(DISTINCT m.user_id) as master_users,
        COUNT(DISTINCT a.raterParticipantId) as agg_users,
        COUNT(DISTINCT CASE WHEN a.raterParticipantId IS NOT NULL THEN m.user_id END) as overlapping_users
    FROM '{master_path}' m
    LEFT JOIN '{aggregates_path}' a ON m.user_id = a.raterParticipantId
""").fetchdf()
print(overlap.to_string(index=False))

# 6. Check period_start overlap
print("\n6️⃣ Checking period_start values:")
master_periods = con.execute(f"""
    SELECT MIN(period_start) as min_period, MAX(period_start) as max_period, COUNT(DISTINCT period_start) as unique_periods
    FROM '{master_path}'
""").fetchdf()
print("Master periods:")
print(master_periods.to_string(index=False))

agg_periods = con.execute(f"""
    SELECT MIN(period_start) as min_period, MAX(period_start) as max_period, COUNT(DISTINCT period_start) as unique_periods
    FROM '{aggregates_path}'
""").fetchdf()
print("\nAggregate periods:")
print(agg_periods.to_string(index=False))

# 7. Try a manual join test
print("\n7️⃣ Manual join test (checking if ANY rows match):")
join_test = con.execute(f"""
    SELECT COUNT(*) as matching_rows
    FROM '{master_path}' m
    INNER JOIN '{aggregates_path}' a 
        ON m.user_id = a.raterParticipantId 
        AND m.period_start = a.period_start
""").fetchdf()
print(join_test.to_string(index=False))

con.close()

print("\n" + "=" * 80)
print("🔍 Debug complete! Check the output above to identify the mismatch.")

🔍 DEBUGGING JOIN ISSUE


1️⃣ user_period_master.parquet columns:
 column_name column_type null  key default extra
     user_id     VARCHAR  YES None    None  None
period_start   TIMESTAMP  YES None    None  None

2️⃣ user_biweekly_aggregates.parquet columns:
       column_name column_type null  key default extra
raterParticipantId     VARCHAR  YES None    None  None
      period_start   TIMESTAMP  YES None    None  None

3️⃣ Sample values from user_period_master:
                                                         user_id period_start user_id_type period_type
313F5490745F282A1FA08CC181256F146CDF7D42015DE340FFB63168F3C56A30   2024-05-09      VARCHAR   TIMESTAMP
4779F711D987B4892A4ECF2AE87CFF6F28D7E382C74840A7126DD578442896C1   2024-03-28      VARCHAR   TIMESTAMP
132977B5FE5A50F4E81664A29B5157F8D88BF654365C97E2DDE9423137571D29   2025-02-27      VARCHAR   TIMESTAMP
033B90CBECD732A561E1FC52BA1C2FA651C221D64DDCE4D1BD4ED8824EB16E69   2024-11-07      VARCHAR   TIMESTAMP
060A613DEA82DF5D8

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 master_users  agg_users  overlapping_users
      1278621    1278615            1278615

6️⃣ Checking period_start values:
Master periods:
min_period max_period  unique_periods
2022-12-22 2025-09-25              73

Aggregate periods:
min_period max_period  unique_periods
2023-01-01 2025-09-21              72

7️⃣ Manual join test (checking if ANY rows match):


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 matching_rows
             0

🔍 Debug complete! Check the output above to identify the mismatch.

🔍 Debug complete! Check the output above to identify the mismatch.


In [4]:
import os, duckdb

# Additional debugging: Check actual data overlap
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data"
master_path = os.path.join(base, "user_period_master.parquet")
aggregates_path = os.path.join(base, "intermediates", "user_biweekly_aggregates.parquet")

con = duckdb.connect()

print("🔍 CHECKING DATA OVERLAP\n")
print("=" * 80)

# 1. Sample actual user IDs from both files
print("\n1️⃣ Sample user IDs from user_period_master:")
master_users = con.execute(f"""
    SELECT DISTINCT user_id 
    FROM '{master_path}' 
    LIMIT 10
""").fetchdf()
print(master_users['user_id'].tolist())

print("\n2️⃣ Sample user IDs from user_biweekly_aggregates:")
agg_users = con.execute(f"""
    SELECT DISTINCT raterParticipantId 
    FROM '{aggregates_path}' 
    LIMIT 10
""").fetchdf()
print(agg_users['raterParticipantId'].tolist())

# 3. Check if ANY of the master users exist in aggregates
print("\n3️⃣ Checking if ANY master users exist in aggregates:")
sample_overlap = con.execute(f"""
    WITH master_sample AS (
        SELECT DISTINCT user_id FROM '{master_path}' LIMIT 100
    )
    SELECT 
        COUNT(DISTINCT m.user_id) as sample_master_users,
        COUNT(DISTINCT a.raterParticipantId) as matching_in_aggregates
    FROM master_sample m
    LEFT JOIN '{aggregates_path}' a ON m.user_id = a.raterParticipantId
    WHERE a.raterParticipantId IS NOT NULL
""").fetchdf()
print(sample_overlap.to_string(index=False))

# 4. Check if ANY of the aggregate users exist in master
print("\n4️⃣ Checking if ANY aggregate users exist in master:")
reverse_overlap = con.execute(f"""
    WITH agg_sample AS (
        SELECT DISTINCT raterParticipantId FROM '{aggregates_path}' LIMIT 100
    )
    SELECT 
        COUNT(DISTINCT a.raterParticipantId) as sample_agg_users,
        COUNT(DISTINCT m.user_id) as matching_in_master
    FROM agg_sample a
    LEFT JOIN '{master_path}' m ON a.raterParticipantId = m.user_id
    WHERE m.user_id IS NOT NULL
""").fetchdf()
print(reverse_overlap.to_string(index=False))

# 5. Check if it's a timestamp format issue
print("\n5️⃣ Checking period_start formats (first few rows):")
master_periods_sample = con.execute(f"""
    SELECT DISTINCT period_start, typeof(period_start) as type
    FROM '{master_path}' 
    ORDER BY period_start
    LIMIT 5
""").fetchdf()
print("Master periods:")
print(master_periods_sample.to_string(index=False))

agg_periods_sample = con.execute(f"""
    SELECT DISTINCT period_start, typeof(period_start) as type
    FROM '{aggregates_path}' 
    ORDER BY period_start
    LIMIT 5
""").fetchdf()
print("\nAggregate periods:")
print(agg_periods_sample.to_string(index=False))

con.close()

print("\n" + "=" * 80)
print("💡 If 'matching_in_aggregates' and 'matching_in_master' are both 0,")
print("   then the two files contain DIFFERENT sets of users!")
print("=" * 80)

🔍 CHECKING DATA OVERLAP


1️⃣ Sample user IDs from user_period_master:
['B96DBA35865FF3ACD48CE1B3E71E46BEB5342A1528953D3CEE7369E65D9B9BE4', 'BDDD6DB726B0B6FF2BB8B718778703C815E13DE6EC971B1F83233213E2D2E75E', 'BE7384DA2B162F0025C298872A68393CAEFDE803EE3DE74FBF17D1D6933C68A2', 'C16FFFAFD367FC48206B552A9496A436C6D7933FA2CFE6379777F3807DF9EE1F', 'C27C4C890FE835711E9BF5659A6371A798FCCCEDC6C712678696F343A924A7BF', 'C3F8C7BFD95EC6B1C0B77AFCE71FFCAB1784822BAFEB9272A2469D1EC1F91D0C', 'CD1E7944DE5F28754EF7FCC6D448E8265206E491C51BA870E6D7691AFD5CAECB', 'CF4E83EE1193644593E348E6EA8DCAC013D4EF98D1FBB7F7C9350FB19F222AC0', 'D3C370C2ED5561692BBD7CFCBEB04AAFCA9CE83E25DA1745B96E126538212479', 'D72D2BDFFE09B8CD8F31E6C696F8ED22D97E289A38E17A39CE42A809B3C3AF8D']

2️⃣ Sample user IDs from user_biweekly_aggregates:
['077228CC77E5F3408DD196BBBD2C4A6EC646E7FAFA75C4B023A90347463989D3', '07728C139050BD52E936E4934624D82703E53B6D8C824C4699DD5E7D9BE3F829', '0773CDBE39E914769C5EC15863D91E502E0ABE8331AC7551BBA0C130F8

In [ ]:
import os, duckdb

# Focus on period_start matching
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data"
master_path = os.path.join(base, "user_period_master.parquet")
aggregates_path = os.path.join(base, "intermediates", "user_biweekly_aggregates.parquet")

con = duckdb.connect()

print("🔍 CHECKING PERIOD_START OVERLAP IN DETAIL\n")
print("=" * 80)

# 1. Get actual period_start values from master
print("\n1️⃣ Sample period_start values from user_period_master:")
master_periods = con.execute(f"""
    SELECT DISTINCT period_start 
    FROM '{master_path}' 
    ORDER BY period_start
    LIMIT 10
""").fetchdf()
print(master_periods['period_start'].tolist())

print("\n2️⃣ Sample period_start values from user_biweekly_aggregates:")
agg_periods = con.execute(f"""
    SELECT DISTINCT period_start 
    FROM '{aggregates_path}' 
    ORDER BY period_start
    LIMIT 10
""").fetchdf()
print(agg_periods['period_start'].tolist())

# 3. Check if ANY periods overlap
print("\n3️⃣ Checking if ANY periods from master exist in aggregates:")
period_overlap = con.execute(f"""
    WITH master_periods AS (
        SELECT DISTINCT period_start FROM '{master_path}'
    ),
    agg_periods AS (
        SELECT DISTINCT period_start FROM '{aggregates_path}'
    )
    SELECT 
        COUNT(DISTINCT m.period_start) as master_unique_periods,
        COUNT(DISTINCT a.period_start) as agg_unique_periods,
        COUNT(DISTINCT CASE WHEN a.period_start IS NOT NULL THEN m.period_start END) as overlapping_periods
    FROM master_periods m
    LEFT JOIN agg_periods a ON m.period_start = a.period_start
""").fetchdf()
print(period_overlap.to_string(index=False))

# 4. Check for exact timestamp matching on a specific user
print("\n4️⃣ Testing join on a specific user with period matching:")
specific_user_test = con.execute(f"""
    WITH sample_user AS (
        SELECT user_id FROM '{master_path}' LIMIT 1
    )
    SELECT 
        m.user_id,
        m.period_start as master_period,
        a.period_start as agg_period,
        m.period_start = a.period_start as periods_match,
        a.helpful_count
    FROM '{master_path}' m
    CROSS JOIN sample_user s
    LEFT JOIN '{aggregates_path}' a 
        ON m.user_id = a.raterParticipantId 
        AND m.period_start = a.period_start
    WHERE m.user_id = s.user_id
    LIMIT 5
""").fetchdf()
print(specific_user_test.to_string(index=False))

# 5. Check if there's a CAST issue with timestamps
print("\n5️⃣ Testing with CAST to DATE:")
cast_test = con.execute(f"""
    SELECT COUNT(*) as matching_rows_with_cast
    FROM '{master_path}' m
    INNER JOIN '{aggregates_path}' a 
        ON m.user_id = a.raterParticipantId 
        AND CAST(m.period_start AS DATE) = CAST(a.period_start AS DATE)
    LIMIT 10
""").fetchdf()
print(cast_test.to_string(index=False))

con.close()

print("\n" + "=" * 80)
print("💡 If overlapping_periods = 0, the period values don't match between files!")
print("=" * 80)